In [1]:
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.model_selection import train_test_split

# Load dataset
# train_path = "/kaggle/input/off-dataset/train.json"
# test_path = "/kaggle/input/off-dataset/test.json"

train_path = r"D:\major project\data\train_final.json"
test_path = r"D:\major project\nepali-offensive-lang-detection-dataset\test.json"

train_df = pd.read_json(train_path)
test_df = pd.read_json(test_path)


# Show basic structure
print("Train shape:", train_df.shape)
print("Test shape:", test_df.shape)
print("\nSample data train:\n")
train_df.head()






Train shape: (5577, 4)
Test shape: (1450, 4)

Sample data train:



,ID,Comment,Label_Binary,Label_Multiclass
0,visewu,भूकने कूकूर ले कहीले टोकदैन भन्छ हो रैछ साला ध...,OFF,OR
1,hgpmyz,तेरी आमाको पसल मूजी कूकूरको मूत खाएसी जे पनी स...,OFF,OO
2,pwqyku,thukka ta machikne valu kun level ko hosh dekh...,OFF,OO
3,voaytw,ग्याँत अग्याँत मूजीहर्लाई ठूऊउउलो मूजी भन्न चा...,OFF,OO
4,mozgzm,गाईलाई गधा भनेर बीराेध गरेकाे छैन हाेला क्यारे,NOFF,NO


In [2]:
print(train_df.head())

       ID                                            Comment Label_Binary  \
0  visewu  भूकने कूकूर ले कहीले टोकदैन भन्छ हो रैछ साला ध...          OFF   
1  hgpmyz  तेरी आमाको पसल मूजी कूकूरको मूत खाएसी जे पनी स...          OFF   
2  pwqyku  thukka ta machikne valu kun level ko hosh dekh...          OFF   
3  voaytw  ग्याँत अग्याँत मूजीहर्लाई ठूऊउउलो मूजी भन्न चा...          OFF   
4  mozgzm     गाईलाई गधा भनेर बीराेध गरेकाे छैन हाेला क्यारे         NOFF   

  Label_Multiclass  
0               OR  
1               OO  
2               OO  
3               OO  
4               NO  


In [3]:
from sklearn.model_selection import train_test_split
# Create validation set (10–15% split from train set)
# train_df, val_df = train_test_split(
#     train_df, test_size=0.15, stratify=train_df["Label_Multiclass"], random_state=42
# )
val_df = pd.read_json('D:/major project/data/val_final.json')
print(f"\nNew train size: {len(train_df)}, Validation size: {len(val_df)}, Test size: {len(test_df)}")


New train size: 5577, Validation size: 620, Test size: 1450


In [21]:
# ! pip install gensim==4.3.1
# ! pip install emoji


In [1]:
import re
import emoji
import pandas as pd
import numpy as np
import regex as regex
import torch
import torch.nn as nn
import torch.optim as optim
from torch.utils.data import Dataset, DataLoader
from torch.nn.utils.rnn import pad_sequence, pack_padded_sequence, pad_packed_sequence
from indic_transliteration import sanscript
from indic_transliteration.sanscript import transliterate
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.linear_model import LogisticRegression
from sklearn.naive_bayes import MultinomialNB
from sklearn.svm import LinearSVC
from sklearn.metrics import classification_report, confusion_matrix, f1_score, accuracy_score
from sklearn.preprocessing import LabelEncoder
from sklearn.utils.class_weight import compute_class_weight
from imblearn.over_sampling import SMOTE
import seaborn as sns
import matplotlib.pyplot as plt
from nltk.corpus import stopwords
import nltk

nltk.download('stopwords')

[nltk_data] Downloading package stopwords to
[nltk_data]     C:\Users\HP\AppData\Roaming\nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


True

In [23]:


# --- Setup ---
nepali_stopwords = set([
    "र", "मा", "कि", "भने", "त", "छ", "हो", "लाई", "ले",
    "गरेको", "गर्छ", "गर्छन्", "हुन्", "गरे", "न", "नभएको"
])

dirghikaran_map = {
    # Vowel elongations / replacements (typical dirghikaran)
    "उ": "ऊ",
    "इ": "ई",
    "ऋ": "रि",  # Often replaced this way in Nepali
    "ए": "ऐ",   # More natural elongated vowel
    "अ": "आ",   # If you want to normalize short 'a' to long 'aa'

    # Remove zero-width joiner/non-joiner
    "\u200d": "",
    "\u200c": "",

    # Normalize punctuation marks
    "।": ".",
    "॥": ".",

    # Common vowel signs normalization (optional)
    "ि": "ी",
    "ु": "ू"
}


roman_stopwords = None  # Will be initialized below

# --- Helper Functions ---

def is_devanagari(text: str) -> bool:
    """Detect if text contains any Devanagari characters."""
    return bool(regex.search(r'\p{Devanagari}', text))

def devanagari_to_roman(text: str) -> str:
    try:
        return transliterate(text, sanscript.DEVANAGARI, sanscript.ITRANS)
    except Exception:
        return text

def roman_to_devanagari(text: str) -> str:
    try:
        return transliterate(text, sanscript.ITRANS, sanscript.DEVANAGARI)
    except Exception:
        return text

def normalize_dirghikaran(text: str) -> str:
    for original, replacement in dirghikaran_map.items():
        text = text.replace(original, replacement)
    return text


def clean_text(text: str) -> str:
    if not isinstance(text, str):
        return ""
    text = text.lower()
    text = re.sub(r"http\S+|www\S+", "", text)         # Remove URLs
    text = re.sub(r"@\w+|#\w+", "", text)              # Remove mentions/hashtags
    text = re.sub(r"\d+", "", text)                    # Remove numbers
    text = emoji.replace_emoji(text, replace="")      # Remove emojis
    text = re.sub(r"[^\w\s]", "", text)                # Remove punctuation
    text = re.sub(r"\s+", " ", text).strip()           # Normalize whitespace
    return text

def clean_text_for_transformer(text: str) -> str:
    if not isinstance(text, str):
        return ""
    # Remove URLs, emojis, and excessive whitespace
    text = re.sub(r"http\S+|www\S+", "", text)
    text = emoji.replace_emoji(text, replace="")
    text = re.sub(r"\s+", " ", text).strip()
    return text


def remove_stopwords_devanagari(text: str) -> str:
    """Remove Devanagari stopwords from text."""
    return ' '.join([word for word in text.split() if word not in nepali_stopwords])

def remove_stopwords_roman(text: str) -> str:
    """Remove Romanized stopwords from text."""
    global roman_stopwords
    if roman_stopwords is None:
        # Initialize roman_stopwords lazily to avoid dependency issues
        roman_stopwords = set([devanagari_to_roman(w) for w in nepali_stopwords])
    return ' '.join([word for word in text.split() if word not in roman_stopwords])

# --- Preprocessing Pipelines ---

def preprocess_for_ml_dl(text: str) -> str:
    """
    Preprocess input text for ML/GRU baselines:
    - Remove stopwords (script dependent)
    - Transliterate Devanagari → Roman
    - Clean Roman text
    """
    if not isinstance(text, str):
        return ""

    if is_devanagari(text):
        text = clean_text(text)
        text = remove_stopwords_devanagari(text)
        text = devanagari_to_roman(text)
    else:
        text = clean_text(text)
        text = remove_stopwords_roman(text)

    return text

def preprocess_for_transformer(text: str) -> str:
    """
    Preprocess input text for Transformer (XLM-Roberta):
    - Transliterate Roman → Devanagari if needed
    - Light cleaning only (no normalization, no punctuation stripping)
    """
    if not isinstance(text, str):
        return ""
    
    if not is_devanagari(text):
        text = roman_to_devanagari(text)

    text = clean_text_for_transformer(text)
    return text


# ----------- Apply Preprocessing to Dataset -----------

for df in [train_df, val_df, test_df]:
    # ML/GRU input: Romanized, cleaned, stopword-removed
    df["clean_comment"] = df["Comment"].apply(preprocess_for_ml_dl)
    df["tokens"] = df["clean_comment"].apply(str.split)

    # Transformer input: Devanagari, normalized, lightly cleaned
    df["transformer_input"] = df["Comment"].apply(preprocess_for_transformer)

# ----------- Inspect Preprocessing on Samples -----------
print("🔍 Sample Preprocessed Outputs:\n")
for idx, row in train_df[["Comment", "clean_comment", "transformer_input"]].head(10).iterrows():
    print(f"📝 Original       : {row['Comment']}")
    print(f"📊 ML/GRU Cleaned : {row['clean_comment']}")
    print(f"🤖 XLM-R Input    : {row['transformer_input']}")
    print("-" * 60)


🔍 Sample Preprocessed Outputs:

📝 Original       : भीखारी कांग्रेस का दलाल
📊 ML/GRU Cleaned : bhakhara kagarasa ka dalala
🤖 XLM-R Input    : भीखारी कांग्रेस का दलाल
------------------------------------------------------------
📝 Original       : फील्म हलमा दाईं तपाईंको टाउकोले छेक्यो भन्ने ठीटी फीट छ हाइट तेरो अर्को पाली देखी आफनो औकात हेरेर सीट ली बूझीस् अझै मूजी ईङलीस छाड्छ
📊 ML/GRU Cleaned : phalama halama daI tapaIka Taukala Chakaya bhanana ThaTa phaTa haiTa tara araka pala dakha Aphana aukata harara saTa la bajhasa ajha maja I~Nalasa ChaDaCha
🤖 XLM-R Input    : फील्म हलमा दाईं तपाईंको टाउकोले छेक्यो भन्ने ठीटी फीट छ हाइट तेरो अर्को पाली देखी आफनो औकात हेरेर सीट ली बूझीस् अझै मूजी ईङलीस छाड्छ
------------------------------------------------------------
📝 Original       : हैट बिदेस मा नि राजनिती पढेलेखेका मान्छे त झन positive खराव हुन्छ्न होला किन त सहिला सोधेर मार्‍यो
📊 ML/GRU Cleaned : haTa badasa ma rajanata paDhalakhaka manaCha jhana positive kharava hanaChana hala kana sahala sa

In [24]:
train_df.head(2)

,ID,Comment,Label_Binary,Label_Multiclass,clean_comment,tokens,transformer_input
3670,pqlvty,भीखारी कांग्रेस का दलाल,OFF,OO,bhakhara kagarasa ka dalala,"[bhakhara, kagarasa, ka, dalala]",भीखारी कांग्रेस का दलाल
3587,dbtslw,फील्म हलमा दाईं तपाईंको टाउकोले छेक्यो भन्ने ठ...,OFF,OO,phalama halama daI tapaIka Taukala Chakaya bha...,"[phalama, halama, daI, tapaIka, Taukala, Chaka...",फील्म हलमा दाईं तपाईंको टाउकोले छेक्यो भन्ने ठ...
